In [1]:
import os
os.chdir(r'C:\Users\p2308924\Practical AI (PAI)\Github CA\PAICA1\Datasets')

import pandas as pd

# Customer

In [2]:
customer = pd.read_csv("training-data-customer-features_v1.0.csv")
customer.head()

,country_code,customer_id,mobile_verified,num_orders_last_50days,num_cancelled_orders_last_50days,num_refund_orders_last_50days,total_payment_last_50days,num_associated_customers,first_order_datetime
0,PH,phjr7fpu,True,0,0,0,0.000000,1,2023-02-06 14:31:37
1,PH,phawdv1w,True,0,0,0,1.082516,1,2023-02-07 10:03:22
2,PH,ph9ocrr7,True,0,0,0,0.050195,2,2022-03-08 06:16:09
3,PH,phf8rsec,True,0,0,0,0.000000,3,2021-08-07 13:54:45
4,PH,ph74rzxh,True,0,0,0,0.000000,1,2022-01-28 12:57:22


In [3]:
customer.shape

(2195916, 9)

In [4]:
customer.isnull().sum()

country_code                           0
customer_id                            0
mobile_verified                        0
num_orders_last_50days                 0
num_cancelled_orders_last_50days       0
num_refund_orders_last_50days          0
total_payment_last_50days              0
num_associated_customers               0
first_order_datetime                1240
dtype: int64

# Order

In [8]:
order = pd.read_csv("training-data-order-features_v1.0.csv")
order.head()

,country_code,order_id,collect_type,payment_method,order_value,num_items_ordered,refund_value,order_date
0,PH,cktw-hdnh,delivery,credit card,18.379008,2.0,6.07,2023/04/09
1,PH,s9mi-zzap,delivery,antfinancial gcash,5.387819,4.0,0.00,2023/04/09
2,PH,vgow-r2i6,delivery,payment on delivery,8.526517,4.0,0.00,2023/04/09
3,PH,u7pg-7rn7,delivery,antfinancial gcash,7.452310,6.0,0.00,2023/04/09
4,PH,q6dn-z0p9,delivery,credit card,9.030052,4.0,0.62,2023/04/09


In [10]:
order.shape

(2270509, 8)

In [6]:
order.isnull().sum()

country_code                           0
customer_id                            0
mobile_verified                        0
num_orders_last_50days                 0
num_cancelled_orders_last_50days       0
num_refund_orders_last_50days          0
total_payment_last_50days              0
num_associated_customers               0
first_order_datetime                1240
dtype: int64

# Labels

In [11]:
labels = pd.read_csv("training-labels_v1.0.csv")
labels.head()

,country_code,order_id,customer_id,is_fraud
0,BD,w2lx-myz3,bdpr8uva,0
1,BD,ta7z-r91q,bd59rlzo,0
2,BD,t5af-wgb2,bd6zhjvq,0
3,BD,sibu-9lm4,bd4fv4rb,0
4,BD,we61-omtr,bdzeepq7,0


In [12]:
labels.shape

(2414179, 4)

In [13]:
labels.isnull().sum()

country_code    0
order_id        0
customer_id     0
is_fraud        0
dtype: int64